In [5]:
from py2neo import authenticate, Graph, Relationship
print "a"
g = Graph("http://localhost:7474/db/data/")
print "b"
#g.delete_all()
from py2neo import Node

a
b


In [6]:
import py2neo

NO_OF_DAYS, NO_OF_FLIGHTS, AP_MULTIPLE = 5, 1600, 1

airport_names = ["Dublin", "Paris","Toulouse", "Bordeaux", "London", "Malaga"]
airport_countries = ["Ireland","France","France","France","Britain","Spain"]

if AP_MULTIPLE > 1:
    airport_names = airport_names + [x + "_x" for x in airport_names]
    airport_countries = airport_countries + airport_countries
    print airport_names
    print airport_countries

In [8]:
airport_dict = {}

"""
    1. Create each Airport Node
    2. Add same country relationship to other Airports in the Same_Country
"""
for name, country in zip(airport_names, airport_countries):
    l = airport_dict.get(country, [])
    
    # 1. Create each Airport Node
    node = Node("Airport", name=name)
    g.create(node)
    
    # 2. Add same country relationship to other Airports in the Same_Country
    for other_airports_in_country in l:
        g.create(Relationship(node, "SAME_COUNTRY", other_airports_in_country))
    
    l.append(node)
    airport_dict[country] = l
    

WEEKDAYS = ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]

print "CREATE", ",".join ( [ '({name}:Airport {{name:"{name}"}})'.format(name=a) for a in airport_names])

CREATE (Dublin:Airport {name:"Dublin"}),(Paris:Airport {name:"Paris"}),(Toulouse:Airport {name:"Toulouse"}),(Bordeaux:Airport {name:"Bordeaux"}),(London:Airport {name:"London"}),(Malaga:Airport {name:"Malaga"})


In [9]:

try:
    g.schema.create_uniqueness_constraint("Airport", "name")
except:
    pass

try:
    g.schema.create_uniqueness_constraint("Weekday", "weekday")
except:
    pass

try:
    g.schema.create_uniqueness_constraint("AirportDay", "date")
except:
    pass

In [10]:
import itertools

days = [ Node("Date", date = i) for i in range (1,NO_OF_DAYS+1)]

"""
    Also need CONNECTING, SHORT, MEDIUM, LONG, WEEKEND
"""
processed_days = []
for day in days:
    g.create( day )
    
    for pd in processed_days:
            """print pd
            print day
            print"""
            g.create( Relationship (pd, "Later", day) )
            
    processed_days.append(day)

In [11]:
airports = list(g.find("Airport"))
airport_days = {}

for airport in airports:
    airport_name = airport['name']
    
    # Add numbers to represent dates
    for day in days:
        ad_name = data = airport_name + str(day['date'])
        ad = Node ("AirportDay", ad_name)
        g.create( ad )
        g.create( Relationship( airport, "AP", ad) )
        g.create( Relationship( day, "DA", ad) )
        airport_days[ad_name] = ad
            
for day in WEEKDAYS:
    g.create ( Node ( "Weekday" , day) )

In [12]:
print airport_days

{u'Malaga4': (a327c86:AirportDay:Malaga4), u'Malaga5': (b6a4112:AirportDay:Malaga5), u'Malaga2': (a29b5f1:AirportDay:Malaga2), u'Malaga3': (a7d96d0:AirportDay:Malaga3), u'Malaga1': (e05c89e:AirportDay:Malaga1), u'Bordeaux5': (d635aeb:AirportDay:Bordeaux5), u'Bordeaux4': (aeacc16:AirportDay:Bordeaux4), u'Bordeaux3': (b3b46c7:AirportDay:Bordeaux3), u'Bordeaux2': (b98acba:AirportDay:Bordeaux2), u'Bordeaux1': (def96ca:AirportDay:Bordeaux1), u'Paris1': (d20ca0f:AirportDay:Paris1), u'Paris3': (c73d637:AirportDay:Paris3), u'Paris2': (a5c8dac:AirportDay:Paris2), u'Paris5': (aeb3292:AirportDay:Paris5), u'Paris4': (c9e8a8d:AirportDay:Paris4), u'London1': (c776789:AirportDay:London1), u'London3': (ddd2a62:AirportDay:London3), u'London2': (ac57794:AirportDay:London2), u'London5': (db16cb8:AirportDay:London5), u'London4': (dc2fc3f:AirportDay:London4), u'Dublin1': (b0afcbf:AirportDay:Dublin1), u'Dublin3': (f229d96:AirportDay:Dublin3), u'Dublin2': (ea5591f:AirportDay:Dublin2), u'Dublin5': (a2673a9:Ai

In [13]:
weekdays = list(g.find("Weekday"))

for ai in airports:
    print ai

(dublin:Airport {name:"Dublin"})
(paris:Airport {name:"Paris"})
(toulouse:Airport {name:"Toulouse"})
(bordeaux:Airport {name:"Bordeaux"})
(london:Airport {name:"London"})
(malaga:Airport {name:"Malaga"})


In [14]:
airports[0].relationships()

()

In [15]:
import random
potential_flights = list ( itertools.permutations ( airport_names , 2) )
random.seed(117)

 

def create_flights():
    
    items = []
    
    

    for i in range(NO_OF_FLIGHTS):

        cost = random.randint(20,100)

        a = random.randint(1, NO_OF_DAYS)

        departure, arrival = random.choice(potential_flights)
        #to = g.find_one("AirportDay", 'data',departure+str(a))
        #dest = g.find_one("AirportDay", 'data', arrival+str(a))
        to = airport_days[departure+str(a)]
        dest = airport_days[arrival+str(a)]
        
        
        if i % 160 == 159:
            print a,to,dest
        weekday = random.choice(weekdays)
        flight_name = departure[:2]+'-'+arrival[:2]+'-'+str(a)
        flight_node = Node("Flight", flight_name = flight_name, flight_cost = cost)
        out, landing = Relationship(to, "Flight_Out", flight_node), Relationship(flight_node, "Flight_In", dest)
        
        items.extend([flight_node, out, landing])

        #g.create(flight_node )
        #g.create(out)
        #g.create(landing)
        
    return items
    
def commit_flights(items):
        
    tx = g.begin()
    for f in items:
        try:
            tx.create(f)
        except e:
            print e
            tx.rollback()
            break
    else:    
        tx.commit()
  

In [16]:
%prun items = create_flights()

2 (b98acba:AirportDay:Bordeaux2) (c294684:AirportDay:Toulouse2)
4 (e266eb5:AirportDay:Toulouse4) (a327c86:AirportDay:Malaga4)
3 (b3b46c7:AirportDay:Bordeaux3) (efce36a:AirportDay:Toulouse3)
1 (fb17c6d:AirportDay:Toulouse1) (b0afcbf:AirportDay:Dublin1)
3 (a7d96d0:AirportDay:Malaga3) (c73d637:AirportDay:Paris3)
2 (a29b5f1:AirportDay:Malaga2) (c294684:AirportDay:Toulouse2)
1 (fb17c6d:AirportDay:Toulouse1) (c776789:AirportDay:London1)
5 (d635aeb:AirportDay:Bordeaux5) (c3fe24c:AirportDay:Toulouse5)
5 (aeb3292:AirportDay:Paris5) (a2673a9:AirportDay:Dublin5)
2 (b98acba:AirportDay:Bordeaux2) (c294684:AirportDay:Toulouse2)
 

In [17]:
%prun commit_flights(items)

In [ ]:
%%time
# Airport connected by AP to AirportDay
flights_from_malaga = """
MATCH (malaga:Airport) - [:AP] - (ad1:AirportDay) - [out1:Flight_Out] - (f:Flight) - [in1:Flight_In] - (ad2:AirportDay) - 
[:AP] - (dest:Airport)
,
(dest:Airport) - [:AP] - (ad3:AirportDay) - [out2:Flight_Out] - (f2:Flight) - 
[in2:Flight_In] - (ad4:AirportDay) - [:AP] - (malaga)
,
(ad2)-[da:DA]-(d1:Date)-[:Later]->(d2:Date)-[:DA] - (ad3)
WHERE malaga.name='Malaga' 
RETURN  *
"""
ffm = list(g.run(flights_from_malaga))
print "NO OF FLIGHTS", NO_OF_FLIGHTS
print "NO OF AIRPORTS"
print "NO OF RESULTS", len(ffm)

In [ ]:
#
from itertools import groupby

results = list(g.run(flights_from_malaga))

print "There are {0} results \n" .format ( str(len(results)) ) 

for r in results:
    for k in r:
        print k
        
    print "\n================\n"

import operator

results_dict = {}

for key, group in groupby(results, lambda result: result['dest']['name']):
    key_dict = results_dict.get(key, [])
    key_dict.extend( group )
    results_dict[key] = key_dict
    
for key in results_dict.keys():
    print key, min ( results_dict[key] , key = operator.itemgetter(1) )['f']['flight_cost']

There are 4887 results 

(e05c89e:AirportDay:Malaga1)
(c776789:AirportDay:London1)
(ac57794:AirportDay:London2)
(a29b5f1:AirportDay:Malaga2)
(e3f30f9:Date {date:1})
(fdad036:Date {date:2})
(e3f30f9)-[:DA]->(c776789)
(london:Airport {name:"London"})
(da92404:Flight {flight_cost:36,flight_name:"Ma-Lo-1"})
(be4f3c6:Flight {flight_cost:80,flight_name:"Lo-Ma-2"})
(da92404)-[:Flight_In]->(c776789)
(be4f3c6)-[:Flight_In]->(a29b5f1)
(malaga:Airport {name:"Malaga"})
(e05c89e)-[:Flight_Out]->(da92404)
(ac57794)-[:Flight_Out]->(be4f3c6)


(e05c89e:AirportDay:Malaga1)
(c776789:AirportDay:London1)
(ac57794:AirportDay:London2)
(a29b5f1:AirportDay:Malaga2)
(e3f30f9:Date {date:1})
(fdad036:Date {date:2})
(e3f30f9)-[:DA]->(c776789)
(london:Airport {name:"London"})
(c0ab530:Flight {flight_cost:52,flight_name:"Ma-Lo-1"})
(be4f3c6:Flight {flight_cost:80,flight_name:"Lo-Ma-2"})
(c0ab530)-[:Flight_In]->(c776789)
(be4f3c6)-[:Flight_In]->(a29b5f1)
(malaga:Airport {name:"Malaga"})
(e05c89e)-[:Flight_Out]->(c0a

# Not sure why identifiers != name
res = list(g.run("""
MATCH (d1:Date)-[:Later]-(d2:Date)
RETURN *
"""))

for re in res:
    print re

In [30]:
import datetime



timeNow = datetime.datetime.now()
t = timeNow + datetime.timedelta(days=5)
print t.strftime('%d/%m/%Y')

print('\xe2\x82\xac')


05/09/2016
€
